In [1]:
from pathlib import Path

import re

from functools import reduce

from collections import defaultdict

from itertools import chain

In [2]:
data_path = Path.home() / 'workstation' / 'dev' / 'Advent-of-Code-2020' / 'data' / 'day16_input.txt'

In [3]:
data_path.exists()

True

In [4]:
with open(data_path, 'r') as reader:
    ticket_input = reader.read().strip()

In [5]:
validity_conditions, ticket, nearby_tickets = ticket_input.split('\n\n')

In [6]:
condition_list = validity_conditions.split('\n')

In [7]:
# https://mathieularose.com/function-composition-in-python/

def compose(*functions):
    return reduce(lambda f, g: lambda x: f(g(x)), functions, lambda x: x)

In [8]:
def inclusive_range(start, stop, step=1):
    return range(start, (stop+1) if step >= 0 else (stop-1), step)

In [9]:
ticket_fields_valid_nums = defaultdict(set) # A dict to keep track of valid numbers for each field

def get_valid_numbers(input_string):
    def parse_range_strings(str_lst):
        return list(map(int, [string for string in str_lst]))
    def get_set_of_nums_from_range(num_list):
        return set(inclusive_range(*num_list))
    
    regex_pattern = re.compile(r'([0-9]+-[0-9]+)')
    ticket_field, condition_string = input_string.split(': ')
    range_1, range_2 = [string.split('-') for string in re.findall(regex_pattern, condition_string)]
    get_set_of_valid_nums  = compose(get_set_of_nums_from_range, parse_range_strings)
    ticket_fields_valid_nums[ticket_field] = get_set_of_valid_nums(range_1).union(get_set_of_valid_nums(range_2))
    return ticket_fields_valid_nums[ticket_field]

In [10]:
def parse_tickets(ticket_string):
    return [list(map(int, ticket.split(','))) for ticket in ticket_string.split('\n')[1:]]

In [11]:
def is_invalid_num_in_ticket(num_list, invalid_nums):
    return any([num in invalid_nums for num in num_list])

In [12]:
def get_product_of_departure_fields(num_list, ref_dict):
    relevant_indices = [value for key, value in ref_dict.items() if 'departure' in key.lower()]
    return reduce(lambda a,b: a*b, map(lambda i: num_list[i], relevant_indices))

#### Part 1

In [13]:
all_valid_numbers = reduce(
    lambda set_1, set_2: set_1.union(set_2), 
    map(get_valid_numbers, condition_list)
)

In [14]:
invalid_nearby_nums = [num for num in chain(*parse_tickets(nearby_tickets)) if num not in all_valid_numbers]

In [15]:
ticket_scan_error_rate = sum(invalid_nearby_nums)

In [16]:
ticket_scan_error_rate

20058

#### Part 2

In [17]:
list_of_nearby_tickets = parse_tickets(nearby_tickets)

valid_nearby_tickets = [ticket for ticket in list_of_nearby_tickets if not is_invalid_num_in_ticket(ticket, invalid_nearby_nums)]
invalid_nearby_tickets = [ticket for ticket in list_of_nearby_tickets if is_invalid_num_in_ticket(ticket, invalid_nearby_nums)]

In [18]:
assert len(valid_nearby_tickets) + len(invalid_nearby_tickets) == len(list_of_nearby_tickets)

In [19]:
temp_copy = dict(ticket_fields_valid_nums)
index_names_dict = defaultdict(list)
index_num = 0

for field_entries in zip(*valid_nearby_tickets):
    entries = set(field_entries)
    for field_name, valid_entries in temp_copy.items():
        invalid_entries = entries - valid_entries
        if not invalid_entries:
            index_names_dict[index_num].append(field_name)
    index_num += 1

In [20]:
sorted_indices = sorted(index_names_dict, key= lambda key: len(index_names_dict[key]))

In [21]:
sorted_indices

[13, 15, 14, 3, 9, 2, 18, 7, 0, 4, 5, 8, 12, 10, 6, 19, 1, 16, 11, 17]

In [22]:
reference_dict = {}

first_index = sorted_indices[0]
first_col_name = index_names_dict[first_index]
reference_dict[first_col_name[0]] = first_index

visited_names = set(index_names_dict[first_index])

for index in sorted_indices[1:]:
    unvisited_name = set(index_names_dict[index]) - visited_names
    field_name = next(iter(unvisited_name))
    reference_dict[field_name] = index
    visited_names.update(unvisited_name)

In [23]:
reference_dict

{'type': 13,
 'arrival platform': 15,
 'duration': 14,
 'route': 3,
 'departure platform': 9,
 'departure station': 2,
 'departure date': 18,
 'departure location': 7,
 'departure time': 0,
 'departure track': 4,
 'arrival track': 5,
 'price': 8,
 'zone': 12,
 'row': 10,
 'arrival location': 6,
 'wagon': 19,
 'arrival station': 1,
 'class': 16,
 'train': 11,
 'seat': 17}

In [24]:
ticket_nums = list(chain(*parse_tickets(ticket)))

In [25]:
get_product_of_departure_fields(ticket_nums, reference_dict)

366871907221